# Preparing Dataset

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, LSTM, Dense

In [2]:
#read the data
dt = pd.read_csv("./data/word_df.csv")

In [3]:
or_cor = []
un_cor = []
for i in range(len(dt['Ordered'])):
  text = dt['Ordered'][i].replace(".","")
  if len(text.split()) == 3:
    or_cor.append(text)
  text = dt['Unordered'][i]
  if len(text.split()) == 3:
    un_cor.append(text)

In [4]:
df = pd.DataFrame()

In [5]:
df['Ordered'] = or_cor
df['Unordered'] = un_cor

In [6]:
df.head()

,Ordered,Unordered
0,I love dogs,I dogs love
1,She plays tennis,She plays tennis
2,They drink coffee,drink coffee They
3,He eats pizza,pizza He eats
4,We go shopping,shopping We go


In [7]:
df.shape

(1038, 2)

In [8]:
#list for words and text
input_words = []
input_text = []
target_words = []
target_text = []

In [9]:
#split the unordered text and append into list
for i in range(len(df)):
    data = df['Unordered'][i]
    data = data.lower()
    word_list = data.split(' ')
    if len(word_list) <= 3:
      input_text.append(word_list)
    for word in word_list:
      if word not in input_words:
        input_words.append(word)

In [10]:
#split the ordered text and append into list
for i in range(len(df)):
    data = df['Ordered'][i]
    data = data.lower()
    word_list = data.split(' ')
    if len(word_list) <= 3:
      target_text.append(word_list)
    for word in word_list:
      if word not in target_words:
        target_words.append(word)

In [11]:
num_encoders_tokens = len(input_words)
num_decoder_tokens = len(target_words)

In [12]:
print(f'num_encoders_tokens : {num_encoders_tokens}')
print(f'num_decoder_tokens : {num_decoder_tokens}')

num_encoders_tokens : 337
num_decoder_tokens : 337


In [13]:
#sort the list
input_words = sorted(list(input_words))

In [14]:
len(input_words)

337

In [15]:
vocabulary = len(input_words)

In [16]:
#assign the index for each word
input_word_index = dict(
    (char, i) for i, char in enumerate(input_words)
)

In [17]:
reverse_word_index = dict(
    (i, char) for i, char in enumerate(input_words)
)

In [18]:
#create the vector for sentence
encoder_input_data = np.zeros(
    (len(input_text), 3, 337), dtype='float32'
)

decoder_input_data = np.zeros(
    (len(input_text), 3, 337), dtype='float32'
)

decoder_target_data = np.zeros(
    (len(input_text), 3, 337), dtype='float32'
)

In [19]:
decoder_input_data.shape

(1038, 3, 337)

In [20]:
#preparing vector 
for j in range(len(input_text)):
  for i in range(len(input_text[j])):
    value = input_word_index[input_text[j][i]]
    encoder_input_data[j][i][value] = 1

In [21]:
#preparing vector 
for j in range(len(target_text)):
  for i in range(len(target_text[j])):
    value = input_word_index[target_text[j][i]]
    decoder_input_data[j][i][value] = 1

In [22]:
input_word_index.items()

dict_items([('abroad', 0), ('animations', 1), ('anthropology', 2), ('appliances', 3), ('archaeology', 4), ('archery', 5), ('art', 6), ('articles', 7), ('astronomy', 8), ('attend', 9), ('attends', 10), ('attentively', 11), ('bake', 12), ('bakes', 13), ('ballet', 14), ('baseball', 15), ('basketball', 16), ('beautifully', 17), ('bicycles', 18), ('bike', 19), ('bikes', 20), ('biking', 21), ('biology', 22), ('birds', 23), ('birdwatching', 24), ('birthdays', 25), ('blogs', 26), ('boardgames', 27), ('books', 28), ('botany', 29), ('boxing', 30), ('breakfast', 31), ('breaks', 32), ('build', 33), ('buildings', 34), ('builds', 35), ('business', 36), ('cakes', 37), ('calligraphy', 38), ('camping', 39), ('canoeing', 40), ('cards', 41), ('care', 42), ('carefully', 43), ('cars', 44), ('cartoons', 45), ('cats', 46), ('celebrate', 47), ('chemistry', 48), ('chess', 49), ('chocolate', 50), ('chores', 51), ('church', 52), ('cities', 53), ('classes', 54), ('clean', 55), ('climb', 56), ('clothes', 57), ('cl

In [23]:
reverse_word_index.items()

dict_items([(0, 'abroad'), (1, 'animations'), (2, 'anthropology'), (3, 'appliances'), (4, 'archaeology'), (5, 'archery'), (6, 'art'), (7, 'articles'), (8, 'astronomy'), (9, 'attend'), (10, 'attends'), (11, 'attentively'), (12, 'bake'), (13, 'bakes'), (14, 'ballet'), (15, 'baseball'), (16, 'basketball'), (17, 'beautifully'), (18, 'bicycles'), (19, 'bike'), (20, 'bikes'), (21, 'biking'), (22, 'biology'), (23, 'birds'), (24, 'birdwatching'), (25, 'birthdays'), (26, 'blogs'), (27, 'boardgames'), (28, 'books'), (29, 'botany'), (30, 'boxing'), (31, 'breakfast'), (32, 'breaks'), (33, 'build'), (34, 'buildings'), (35, 'builds'), (36, 'business'), (37, 'cakes'), (38, 'calligraphy'), (39, 'camping'), (40, 'canoeing'), (41, 'cards'), (42, 'care'), (43, 'carefully'), (44, 'cars'), (45, 'cartoons'), (46, 'cats'), (47, 'celebrate'), (48, 'chemistry'), (49, 'chess'), (50, 'chocolate'), (51, 'chores'), (52, 'church'), (53, 'cities'), (54, 'classes'), (55, 'clean'), (56, 'climb'), (57, 'clothes'), (58,

In [24]:
decoder_target_data = decoder_input_data.copy()

In [25]:
decoder_target_data[0].shape

(3, 337)

In [26]:
temp = []

In [27]:
#teacher forcing
for i in range(len(decoder_target_data)):
  a = np.insert(decoder_target_data[i], 0, np.zeros(337))
  a = a[:-337]
  a = a.reshape(3, 337)
  temp.append(a)  

In [28]:
temp = np.array(temp)

In [29]:
#[loves she you] -> [she loves you] === [2 1 3] -> [1 2 3]  but decoder takes [0 1 2]
temp[0][1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Decode the data

In [30]:
index = [np.where(decoder_input_data[0][0] == 1)[0][0], np.where(decoder_input_data[0][1] == 1)[0][0], np.where(decoder_input_data[0][2] == 1)[0][0]]
print(index)

[149, 174, 89]


In [31]:
t = []
for i in index:
  t.append(reverse_word_index[i])
t

['i', 'love', 'dogs']

# Model

## Encoder

In [32]:
latent_dim = 256
batch_size = 16
epochs = 16

In [33]:
# Define input sequence
encoder_inputs = Input(shape=(None, num_decoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

## Decoder

In [34]:
# Define decoder sequence
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [35]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [36]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=8, epochs=epochs, validation_split=0.2)

Epoch 1/16
104/104 [==============================] - 2s 15ms/step - loss: 0.4293 - accuracy: 0.9293 - val_loss: 0.7033 - val_accuracy: 0.8734
Epoch 2/16
104/104 [==============================] - 2s 16ms/step - loss: 0.3743 - accuracy: 0.9406 - val_loss: 0.6392 - val_accuracy: 0.8830
Epoch 3/16
104/104 [==============================] - 1s 14ms/step - loss: 0.3234 - accuracy: 0.9490 - val_loss: 0.6513 - val_accuracy: 0.8814
Epoch 4/16
104/104 [==============================] - 2s 15ms/step - loss: 0.2830 - accuracy: 0.9598 - val_loss: 0.5631 - val_accuracy: 0.8894
Epoch 5/16
104/104 [==============================] - 2s 15ms/step - loss: 0.2428 - accuracy: 0.9631 - val_loss: 0.5403 - val_accuracy: 0.9038
Epoch 6/16
104/104 [==============================] - 1s 14ms/step - loss: 0.2085 - accuracy: 0.9687 - val_loss: 0.5312 - val_accuracy: 0.8910
Epoch 7/16
104/104 [==============================] - 1s 14ms/step - loss: 0.1750 - accuracy: 0.9763 - val_loss: 0.5085 - val_accuracy: 0.9103

# Prediction

In [49]:
encoder_model = Model(encoder_inputs, encoder_states)

In [50]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [51]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_encoders_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_seq = list()
    while not stop_condition:

        # in a loop
        # decode the input to a token/output prediction + required states for context vector
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # convert the token/output prediction to a token/output
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_digit = sampled_token_index
        # add the predicted token/output to output sequence
        decoded_seq.append(sampled_digit)
        

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_digit == '\n' or
           len(decoded_seq) == 3):
            stop_condition = True

        # Update the input target sequence (of length 1) 
        # with the predicted token/output 
        target_seq = np.zeros((1, 1, num_encoders_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update input states (context vector) 
        # with the ouputed states
        states_value = [h, c]

        # loop back.....
        
    # when loop exists return the output sequence
    return decoded_seq

In [52]:
predicted= decode_sequence(encoder_input_data[0].reshape(1,3,num_encoders_tokens))

1/1 [==============================] - 0s 15ms/step


In [53]:
predicted

[149, 174, 89]

In [54]:
predicted_text = []
encoded_text = []
index = [np.where(encoder_input_data[0][0] == 1)[0][0], np.where(encoder_input_data[0][1] == 1)[0][0], np.where(encoder_input_data[0][2] == 1)[0][0]]
for i in range(len(predicted)):
  predicted_text.append(reverse_word_index[predicted[i]])
  encoded_text.append(reverse_word_index[index[i]])

In [55]:
print(encoded_text, " ---> ", predicted_text)

['i', 'dogs', 'love']  --->  ['i', 'love', 'dogs']


In [56]:
predicted_text = []
encoded_text = []
sampleNo = 10
for i in range(0, sampleNo-1):
    predicted = decode_sequence(encoder_input_data[i].reshape(1,3,num_encoders_tokens))
    index = [np.where(encoder_input_data[i][0] == 1)[0][0], np.where(encoder_input_data[i][1] == 1)[0][0], np.where(encoder_input_data[i][2] == 1)[0][0]]
    predicted_text.append(predicted)
    encoded_text.append(index)

1/1 [==============================] - 0s 15ms/step


In [57]:
decode_encoded_word = []
decode_predicted_word = []
for encode in encoded_text:
    temp_list = []
    for token in range(len(encode)):
        temp_list.append(reverse_word_index[encode[token]])
    decode_encoded_word.append(temp_list)

for encode in predicted_text:
    temp_list = []
    for token in range(len(encode)):
        temp_list.append(reverse_word_index[encode[token]])
    decode_predicted_word.append(temp_list)

In [58]:
for i in range(len(decode_predicted_word)):
    print(decode_encoded_word[i], ' ---> ', decode_predicted_word[i])

['i', 'dogs', 'love']  --->  ['i', 'love', 'dogs']
['she', 'plays', 'tennis']  --->  ['she', 'plays', 'tennis']
['drink', 'coffee', 'they']  --->  ['they', 'drink', 'coffee']
['pizza', 'he', 'eats']  --->  ['he', 'eats', 'pizza']
['shopping', 'we', 'go']  --->  ['we', 'go', 'shopping']
['gracefully', 'she', 'dances']  --->  ['she', 'dances', 'gracefully']
['runs', 'fast', 'he']  --->  ['he', 'runs', 'fast']
['they', 'daily', 'swim']  --->  ['they', 'swim', 'daily']
['i', 'study', 'hard']  --->  ['i', 'study', 'hard']


# Save the Model

In [60]:
# save the weights individually
for layer in model.layers:
    weights = layer.get_weights()
    if weights != []:
        np.savez(f'./assets/{layer.name}.npz', weights)

C:\Users\Dark-Devil\anaconda3\lib\site-packages\numpy\lib\npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


# Load Model

In [69]:
# Define input sequence
encoder_inputs = Input(shape=(None, num_decoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define decoder sequence
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
loaded_model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='Training_model')

In [70]:
w_encoder_lstm = np.load('./assets/lstm.npz', allow_pickle=True)
w_decoder_lstm = np.load('./assets/lstm_1.npz', allow_pickle=True)
w_dense = np.load('./assets/dense.npz', allow_pickle=True)

In [71]:
# set the weights of the model
loaded_model.layers[2].set_weights(w_encoder_lstm['arr_0'])
loaded_model.layers[3].set_weights(w_decoder_lstm['arr_0'])
loaded_model.layers[4].set_weights(w_dense['arr_0'])

In [72]:
#encoder model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

#decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
predicted= decode_sequence(encoder_input_data[0].reshape(1,3,num_encoders_tokens))